In [1]:
# In your Jupyter notebook (TEST_Daily_Analysis.ipynb)
import sys
from pathlib import Path

# Add the project root directory to Python path
ROOT_DIR = Path.cwd().parent  # Goes up one level from 'scanner' to 'stockbot4'
if str(ROOT_DIR) not in sys.path:
    sys.path.append(str(ROOT_DIR))

In [2]:
from ib_insync import *
util.startLoop()

ib = IB()
ib.connect('127.0.0.1', 7496, clientId=12)

<IB connected to 127.0.0.1:7496 clientId=12>

In [3]:

#! Draft example of how to use the StockX class

import stock
from strategies import ta

sx = stock.StockXDaily(ib, "TSLA")

# getting the stock data
allowed_etfs = ['XLK', 'XLY', 'XLI']
sx.req_fundamentals(max_days_old=1) # gets the fundamentals from the IB API
sx.validate_fundamental('primary_etf',      'isin', allowed_etfs,    description='Stocks primary secteor ETF is allowed')
sx.validate_fundamental('secondary_etf',    'isin', allowed_etfs,    description='Stocks primary secteor ETF is allowed')
sx.validate_fundamental('market_cap',       '>=', 300,               description='Market cap is greater than 300M') 
sx.validate_fundamental('volume_10day_avg', '>=', 0.3,               description='Volume is greater than 300k')
display(sx.validation_fundamentals_report(asDF=True, save_image=True)) # returns a dictionary with the validation results
sx.validation_fundamentals_has_passed(maxFails=0) # returns boolean if all validations are True


Using cached data for TSLA (age: 2024-12-14 09:54:38)


,key,description,validation_type,comparison_value,actual_value,passed
"primary_etf_isin_['XLK', 'XLY', 'XLI']",primary_etf,Stocks primary secteor ETF is allowed,isin,"[XLK, XLY, XLI]","(XLY, 0.56)",True
"secondary_etf_isin_['XLK', 'XLY', 'XLI']",secondary_etf,Stocks primary secteor ETF is allowed,isin,"[XLK, XLY, XLI]","(XLI, 0.44)",True
market_cap_>=_300,market_cap,Market cap is greater than 300M,>=,300,1342126.0,True
volume_10day_avg_>=_0.3,volume_10day_avg,Volume is greater than 300k,>=,0.3,77.30541,True


True

In [4]:
 # gets the ohlcv data from the IB APIs
sx.req_ohlcv()
# price
sx.validate_TA(ta.ColVal('close'),           ta.AboveBelow('CV_close', 'above', 1),                                                                  description='close price is above 1')
sx.validate_TA(ta.MA('close', 200),          ta.AboveBelow('close', 'above', 'MA_cl_200',), {'dash': 'solid', 'color': 'cyan', 'width': 3},   row=1, description='close price is above 200 MA') 
sx.validate_TA(ta.MA('close', 150),          ta.AboveBelow('close', 'above', 'MA_cl_150',), {'dash': 'solid', 'color': 'pink', 'width': 2},   row=1, description='close price is above 150 MA') 
sx.validate_TA(ta.MA('close', 50),           ta.Breaks    ('close', 'above', 'MA_cl_50'),   {'dash': 'solid', 'color': 'purple', 'width': 2}, row=1, description='close price breaks above 50 MA')
sx.validate_TA(ta.PctChange('MA_cl_50', 1),  ta.AboveBelow('PCT_MA_cl_50_1', 'above', 0) ,                                                           description='pct change of 50 MA is above 0')
sx.validate_TA(ta.PctChange('close', 1),     ta.AboveBelow('PCT_close_1', 'above', 4) ,                                                              description='pct change of close is above 4 (4% Gap)')
# volume
sx.validate_TA(ta.ColVal('volume'),          ta.AboveBelow('CV_volume','above', 50_000),                                                             description='volume is above 50k')
sx.validate_TA(ta.MA('volume', 10),          ta.Breaks('volume','above', 'MA_vo_10'), {'dash': 'solid', 'color': 'pink', 'width': 1}, row=2,         description='volume breaks above 10 MA')
sx.validate_TA(ta.VolDev('volume', 10),      ta.AboveBelow('VDEV_10', 'above', 80), {'dash': 'solid', 'color': 'pink', 'width': 1}, row=3,           description='volume is above 80% of 10 MA Deviation')

df = sx.validation_TA_report(asDF=True, save_image=True) # returns a dictionary with the validation results
print(sx.validation_TA_has_passed()) # returns boolean if all validations are True
display(df)



Loading data from data/historical_data_store/TSLA_1_day.csv
Stored data: 255 rows of data
Loading data from data/historical_data_store/TSLA_1_day.csv
False


,description,indicator,validator,validation_type,indicator_value,passed,timestamp
CV_close_AB_CV_close_ab_1,close price is above 1,CV_close,AB_CV_close_ab_1,AboveBelow,436.230000,True,2024-12-13
MA_cl_200_AB_close_ab_MA_cl_200,close price is above 200 MA,MA_cl_200,AB_close_ab_MA_cl_200,AboveBelow,221.944300,True,2024-12-13
MA_cl_150_AB_close_ab_MA_cl_150,close price is above 150 MA,MA_cl_150,AB_close_ab_MA_cl_150,AboveBelow,237.926667,True,2024-12-13
MA_cl_50_BRK_close_ab_MA_cl_50,close price breaks above 50 MA,MA_cl_50,BRK_close_ab_MA_cl_50,Breaks,293.112800,False,2024-12-13
PCT_MA_cl_50_1_AB_PCT_MA_cl_50_1_ab_0,pct change of 50 MA is above 0,PCT_MA_cl_50_1,AB_PCT_MA_cl_50_1_ab_0,AboveBelow,1.203268,True,2024-12-13
PCT_close_1_AB_PCT_close_1_ab_4,pct change of close is above 4 (4% Gap),PCT_close_1,AB_PCT_close_1_ab_4,AboveBelow,4.336283,True,2024-12-13
CV_volume_AB_CV_volume_ab_50000,volume is above 50k,CV_volume,AB_CV_volume_ab_50000,AboveBelow,613606.000000,True,2024-12-13
MA_vo_10_BRK_volume_ab_MA_vo_10,volume breaks above 10 MA,MA_vo_10,BRK_volume_ab_MA_vo_10,Breaks,536521.900000,False,2024-12-13
VDEV_10_AB_VDEV_10_ab_80,volume is above 80% of 10 MA Deviation,VDEV_10,AB_VDEV_10_ab_80,AboveBelow,14.367373,False,2024-12-13


In [5]:
import pandas as pd
import stock
from strategies import ta

# simulated the scan results from the stockbot market scanner
scan_results_df = pd.DataFrame({
    'rank': [1, 2, 3],
    'symbol': ['TSLA', 'AAPL', 'MSFT'],
})

# getting the stock data
allowed_etfs = ['XLK', 'XLY', 'XLI']

list_results = []

for s in scan_results_df['symbol']:
    sx = stock.StockXDaily(ib, s)
    sx.req_fundamentals(max_days_old=1) # gets the fundamentals from the IB API
    validation_results = {
        'Sector1 Valid': sx.validate_fundamental('primary_etf',      'isin', allowed_etfs,    description='Stocks primary secteor ETF is allowed'),
        'Sector2 Valid': sx.validate_fundamental('secondary_etf',    'isin', allowed_etfs,    description='Stocks primary secteor ETF is allowed'),
        'Market Cap > 300M': sx.validate_fundamental('market_cap',     '>=', 300,             description='Market cap is greater than 300M') ,
        'Vol 10DayMA > 300K': sx.validate_fundamental('volume_10day_avg', '>=', 0.3,               description='Volume is greater than 300k')
    }
    list_results.append(validation_results)

df = pd.DataFrame(list_results)
df


Using cached data for TSLA (age: 2024-12-14 09:54:38)
Using cached data for AAPL (age: 2024-12-14 09:55:07)
Using cached data for MSFT (age: 2024-12-15 10:35:00)


,Sector1 Valid,Sector2 Valid,Market Cap > 300M,Vol 10DayMA > 300K
0,True,True,True,True
1,True,True,True,True
2,True,False,True,True


In [6]:
import pandas as pd
import stock
from strategies import ta

# simulated the scan results from the stockbot market scanner
scan_results_df = pd.DataFrame({
    'rank': [1, 2, 3],
    'symbol': ['TSLA', 'AAPL', 'MSFT'],
})

# getting the stock data
allowed_etfs = ['XLK', 'XLY', 'XLI']

list_results = []

for s in scan_results_df['symbol']:
    sx = stock.StockXDaily(ib, s)
    sx.req_ohlcv() # gets the fundamentals from the IB API
    validation_results = {
            'Close > $1'       : sx.validate_TA(ta.ColVal('close'),          ta.AboveBelow('CV_close', 'above', 1),         description='close price is above 1'),
            'Above 200MA'      : sx.validate_TA(ta.MA('close', 200),         ta.AboveBelow('close', 'above', 'MA_cl_200'),  description='close price is above 200 MA',      style={'dash': 'solid', 'color': 'cyan', 'width': 3}, row=1),
            'Above 150MA'      : sx.validate_TA(ta.MA('close', 150),         ta.AboveBelow('close', 'above', 'MA_cl_150'),  description='close price is above 150 MA',      style={'dash': 'solid', 'color': 'pink', 'width': 2}, row=1),
            'Breaks Above 50MA': sx.validate_TA(ta.MA('close', 50),          ta.Breaks('close', 'above', 'MA_cl_50'),       description='close price breaks above 50 MA',   style={'dash': 'solid', 'color': 'purple', 'width': 2}, row=1),
            '50MA Slope > 0'   : sx.validate_TA(ta.PctChange('MA_cl_50', 1), ta.AboveBelow('PCT_MA_cl_50_1', 'above', 0),   description='pct change of 50 MA is above 0'),
            'Gap Up > 4%'      : sx.validate_TA(ta.PctChange('close', 1),    ta.AboveBelow('PCT_close_1', 'above', 4),      description='pct change of close is above 4 (4% Gap)'),
            'Volume > 50K'     : sx.validate_TA(ta.ColVal('volume'),         ta.AboveBelow('CV_volume', 'above', 50_000),   description='volume is above 50k'),
            'Volume Above 10MA': sx.validate_TA(ta.MA('volume', 10),         ta.Breaks('volume', 'above', 'MA_vo_10'),      description='volume breaks above 10 MA',              style={'dash': 'solid', 'color': 'pink', 'width': 1}, row=2),
            'Volume Dev > 80%' : sx.validate_TA(ta.VolDev('volume', 10),     ta.AboveBelow('VDEV_10', 'above', 80),         description='volume is above 80% of 10 MA Deviation', style={'dash': 'solid', 'color': 'pink', 'width': 1}, row=3, ),
            'TA Passed'        : sx.validation_TA_has_passed()
        }
    list_results.append(validation_results)

df = pd.DataFrame(list_results)
df


Loading data from data/historical_data_store/TSLA_1_day.csv
Stored data: 255 rows of data
Loading data from data/historical_data_store/TSLA_1_day.csv
Loading data from data/historical_data_store/AAPL_1_day.csv
Stored data: 255 rows of data
Loading data from data/historical_data_store/AAPL_1_day.csv
Loading data from data/historical_data_store/MSFT_1_day.csv
Stored data: 255 rows of data
Loading data from data/historical_data_store/MSFT_1_day.csv


,Close > $1,Above 200MA,Above 150MA,Breaks Above 50MA,50MA Slope > 0,Gap Up > 4%,Volume > 50K,Volume Above 10MA,Volume Dev > 80%,TA Passed
0,True,True,True,False,True,True,True,False,False,False
1,True,True,True,False,True,False,True,False,False,False
2,True,True,True,False,True,False,True,False,False,False


In [7]:
import pandas as pd
import stock
from strategies import ta

def update_scan_results(scan_results_df, allowed_etfs, ib):
    for s in scan_results_df['symbol']:
        sx = stock.StockXDaily(ib, s)
        sx.req_fundamentals(max_days_old=1)  # gets the fundamentals from the IB API
        sx.req_ohlcv()
        fund_results = sx.get_funadmentals_validation_results(allowed_etfs)
        ta_results   = sx.get_TA_validation_results()

        # Find the index of the row with the symbol
        index = scan_results_df[scan_results_df['symbol'] == s].index[0]

        # Update the row with the validation results
        for key, value in fund_results.items():
            scan_results_df.loc[index, key] = value
        
        # Update the row with the validation results
        for key, value in ta_results.items():
            scan_results_df.loc[index, key] = value

    return scan_results_df

# Example usage
scan_results_df = pd.DataFrame({
    'rank': [1, 2, 3],
    'symbol': ['TSLA', 'AAPL', 'MSFT'],
})



updated_scan_results_df = update_scan_results(scan_results_df, ['XLK', 'XLY', 'XLI'], ib)
updated_scan_results_df

Using cached data for TSLA (age: 2024-12-14 09:54:38)
Loading data from data/historical_data_store/TSLA_1_day.csv
Stored data: 255 rows of data
Loading data from data/historical_data_store/TSLA_1_day.csv
Using cached data for AAPL (age: 2024-12-14 09:55:07)
Loading data from data/historical_data_store/AAPL_1_day.csv
Stored data: 255 rows of data
Loading data from data/historical_data_store/AAPL_1_day.csv
Using cached data for MSFT (age: 2024-12-15 10:35:00)
Loading data from data/historical_data_store/MSFT_1_day.csv
Stored data: 255 rows of data
Loading data from data/historical_data_store/MSFT_1_day.csv


,rank,symbol,Sector1 Valid,Sector2 Valid,Market Cap > 300M,Vol 10DayMA > 300K,Fundamentals Passed,Close > $1,Above 200MA,Above 150MA,Breaks Above 50MA,50MA Slope > 0,Gap Up > 4%,Volume > 50K,Volume Above 10MA,Volume Dev > 80%,TA Passed
0,1,TSLA,True,True,True,True,True,True,True,True,False,True,True,True,False,False,False
1,2,AAPL,True,True,True,True,True,True,True,True,False,True,False,True,False,False,False
2,3,MSFT,True,False,True,True,False,True,True,True,False,True,False,True,False,False,False


In [8]:
ib.disconnect()

In [5]:
sx.frame.setup_chart()
sx.frame.plot() # plots the stock data  
sx.save_chart() # saves the chart to a file
sx.save_zoomed_chart(show=True) # shows the chart in the notebook


Chart saved as TSLA.png
Zoomed chart saved as TSLA_zoomed.png


In [6]:
sx.email_report()

Email sent to for TSLA


In [7]:

# scoring the stock
sx.score_fundamantal('pe_ratio', 10, 20) # returns a score between 0 and 100
sx.score_fundamantal('consensus_recommendation', 1, 20) # returns a score between 0 and 100
sx.score_fundamantal_report() # returns a dictionary with the scoring results
sx.score_fundamantal_has_passed(minScore=20, minAvScore=50) # returns boolean if all scores are True

# scoring the technical analysis
sx.score_TA(ta.MansfieldRSI('close', 'SPY', 200),  0, 2) # returns a score between 0 and 100
sx.score_TA(ta.MansfieldRSI('close', 'XLK', 200),  0, 2) # returns a score between 0 and 100
sx.score_TA(ta.PivotsBroken('HP1'),  0, 2) # returns a score between 0 and 100
sx.score_TA(ta.RoomToMove('Res1'),  0, 2) # returns a score between 0 and 100
sx.score_TA(ta.VolAcc(0.0001),  0, 100) # returns a score between 0 and 100
sx.score_TA_report() # returns a dictionary with the scoring results
sx.score_TA_has_passed(minScore=20, minAvScore=50) # returns boolean if all scores are True

AttributeError: 'StockXDaily' object has no attribute 'score_fundamantal'

In [ ]:
import stock
sx = stock.StockXDaily(ib, "TSLA")
sx.req_fundamentals()

{'industry': 'Consumer, Cyclical',
 'category': 'Auto Manufacturers',
 'subcategory': 'Auto-Cars/Light Trucks',
 'primary_etf': ('XLY', 0.56),
 'secondary_etf': ('XLI', 0.44),
 'currency': 'USD',
 'longName': 'TESLA INC',
 'timeZoneId': 'US/Eastern',
 'tradingHours': ('0400', '2000'),
 'liquidHours': ('0930', '1600'),
 'current_price': 389.79,
 'high_52week': 404.8,
 'low_52week': 138.8025,
 'pricing_date': '2024-12-09T00:00:00',
 'volume_10day_avg': 69.95339,
 'enterprise_value': 1261250.0,
 'market_cap': 1287202.0,
 'revenue_ttm': 97150.0,
 'ebitda': 13338.0,
 'net_income_ttm': 12743.0,
 'eps_ttm': 3.64717,
 'revenue_per_share': 27.83867,
 'book_value_per_share': 21.80574,
 'cash_per_share': 10.49205,
 'cash_flow_per_share': 5.12014,
 'dividend_per_share': 0.0,
 'gross_margin': 18.22851,
 'operating_margin': 0.0,
 'net_profit_margin': 0.0,
 'revenue_growth_rate': 0.0,
 'eps_growth_rate': 0.0,
 'pe_ratio': 109.9455,
 'price_to_book': 18.3892,
 'price_to_sales': 0.0,
 'employee_count':

In [ ]:

sx.req_fundamentals()
sx.fundamentals.__dict__

In [ ]:
sx.validate_fundamental

In [ ]:
tsla.get_daily_data()

Loading data from data/historical_data_store/TSLA_1_day.csv
Stored data: 250 rows of data
Loading data from data/historical_data_store/TSLA_1_day.csv


,open,high,low,close,volume
date,,,,,
2023-12-06,240.61,246.57,238.85,240.37,1080540
2023-12-07,240.80,244.08,236.98,242.91,916952
2023-12-08,242.00,245.27,239.27,243.23,878583
2023-12-11,242.00,245.00,237.45,239.55,819699
2023-12-12,240.25,241.22,233.87,236.48,785221
...,...,...,...,...,...
2024-11-26,338.00,346.96,335.21,340.17,470824
2024-11-27,339.76,342.60,326.59,334.40,441879
2024-11-29,336.00,348.50,334.65,345.44,272292


In [ ]:
tsla.set_default_daily_ta()
tsla.frame.data

,open,high,low,close,volume,MA_cl_200,MA_cl_50,MA_vo_10,ACC_close,VDEV_10,TDUR_MA_cl_50
date,,,,,,,,,,,
2023-12-06,240.61,246.57,238.85,240.37,1080540,NaN,NaN,NaN,NaN,NaN,1
2023-12-07,240.80,244.08,236.98,242.91,916952,NaN,NaN,NaN,NaN,NaN,1
2023-12-08,242.00,245.27,239.27,243.23,878583,NaN,NaN,NaN,NaN,NaN,1
2023-12-11,242.00,245.00,237.45,239.55,819699,NaN,NaN,NaN,NaN,NaN,1
2023-12-12,240.25,241.22,233.87,236.48,785221,NaN,NaN,NaN,NaN,NaN,1
...,...,...,...,...,...,...,...,...,...,...,...
2024-11-26,338.00,346.96,335.21,340.17,470824,213.43710,269.0762,723829.0,65.76,-34.953698,45
2024-11-27,339.76,342.60,326.59,334.40,441879,214.18910,271.1760,671255.0,48.25,-34.171217,46
2024-11-29,336.00,348.50,334.65,345.44,272292,214.96490,273.2024,605475.7,21.40,-55.028418,47


In [ ]:

tsla.compute_daily_filters_with_scores()


,BRK_UP_MA,BRK_UP_MA,ABV_close_MA_cl_50,ABV_close_MA_cl_200,ABV_3_TDUR_MA_cl_50,ABV_80_VDEV_10,filter_score,all_true
date,,,,,,,,
2023-12-06,False,False,False,False,True,False,1,False
2023-12-07,False,False,False,False,True,False,1,False
2023-12-08,False,False,False,False,True,False,1,False
2023-12-11,False,False,False,False,True,False,1,False
2023-12-12,False,False,False,False,True,False,1,False
...,...,...,...,...,...,...,...,...
2024-11-26,False,False,True,True,False,True,3,False
2024-11-27,False,False,True,True,False,True,3,False
2024-11-29,False,False,True,True,False,True,3,False


In [ ]:
ib.disconnect()

In [ ]:
from data import historical_data as hd
from ib_insync import *


tdata = hd.get_hist_data('TSLA', "52 weeksAgo", 'now', '1 day')
tdata

Loading data from data/historical_data_store/TSLA_1_day.csv
Stored data: 250 rows of data
Loading data from data/historical_data_store/TSLA_1_day.csv


,open,high,low,close,volume
date,,,,,
2023-12-06,240.61,246.57,238.85,240.37,1080540
2023-12-07,240.80,244.08,236.98,242.91,916952
2023-12-08,242.00,245.27,239.27,243.23,878583
2023-12-11,242.00,245.00,237.45,239.55,819699
2023-12-12,240.25,241.22,233.87,236.48,785221
...,...,...,...,...,...
2024-11-26,338.00,346.96,335.21,340.17,470824
2024-11-27,339.76,342.60,326.59,334.40,441879
2024-11-29,336.00,348.50,334.65,345.44,272292


In [ ]:
mansfield, roc, combined = analyze_sector(
    etf_symbol='XLE',
    lookback_period="26 weeksAgo",
    mansfield_period=150,
    roc_period=150,
    ma_short=20,
    ma_long=100,
    verbose=True
)

In [ ]:

    



# Example Usage
# Assuming `etf_data` and `market_data` are DataFrames with the required columns


from data import historical_data as hd
import compare



etf_data = hd.get_hist_data('XLF', "52 weeksAgo", 'now', '1 day')
market_data = hd.get_hist_data('SPY', "52 weeksAgo", 'now', '1 day')

analysis = compare.SectorAnalysis(etf_data, market_data)
analysis.compute_all(
    mansfield_period=200,  # Period for Mansfield RSI
    roc_period=200,         # Period for MA ROC ratio
    ma_short=50,           # Short MA period
    ma_long=200           # Long MA period
)
result_df = analysis.get_df()
today_metrics = analysis.get_today(verbose=True)
display(today_metrics)
result_df


Loading data from data/historical_data_store/XLF_1_day.csv
Stored data: 249 rows of data
Loading data from data/historical_data_store/XLF_1_day.csv
Loading data from data/historical_data_store/SPY_1_day.csv
Stored data: 250 rows of data
Loading data from data/historical_data_store/SPY_1_day.csv

=== Today's Market Analysis ===

Mansfield RSI: 4.25
MA ROC Ratio: 1.16
Combined Score: 2.71

📈 BULLISH


mansfield_rsi     4.254055
ma_roc_ratio      1.160618
combined_score    2.707337
dtype: float64

,etf_50_ma,etf_200_ma,market_50_ma,market_200_ma,mansfield_rsi,ma_roc_ratio,combined_score
date,,,,,,,
2024-09-25,43.9020,41.07935,551.0202,520.40360,-0.554947,0.000000,-0.277474
2024-09-26,43.9246,41.12345,551.3066,520.96265,-0.521474,-0.003399,-0.262437
2024-09-27,43.9616,41.16730,551.6480,521.50450,0.199075,0.125981,0.162528
2024-09-30,44.0124,41.21035,552.1318,522.04710,0.014426,0.025382,0.019904
2024-10-01,44.0514,41.24860,552.4014,522.52575,0.510626,0.061587,0.286106
2024-10-02,44.0928,41.28670,552.7696,523.01615,0.406175,-0.079116,0.163530
2024-10-03,44.1360,41.32415,553.2764,523.50465,0.084966,-0.144174,-0.029604
2024-10-04,44.1912,41.36600,553.9360,524.00725,1.017609,0.274168,0.645889
2024-10-07,44.2238,41.40340,554.4046,524.47240,0.609759,0.092630,0.351194


In [ ]:
etf_data

,open,high,low,close,volume
date,,,,,
2023-12-08,35.97,36.19,35.83,36.14,311722
2023-12-11,36.13,36.42,36.09,36.38,287274
2023-12-12,36.35,36.66,36.29,36.66,298369
2023-12-13,36.69,37.41,36.55,37.41,525548
2023-12-14,37.28,37.71,37.23,37.55,622257
...,...,...,...,...,...
2024-11-27,51.26,51.62,51.18,51.33,289386
2024-11-29,51.51,51.57,51.21,51.26,138172
2024-12-02,51.35,51.49,50.70,50.86,477889


In [ ]:
from ib_insync import IB, Stock
import xml.etree.ElementTree as ET

contract = Stock('TSLA', 'SMART', 'USD')

# fundamental_data = ib.reqFundamentalData(contract, 'ReportsFundamentals')
fundamental_data = ib.reqFundamentalData(contract, 'ReportSnapshot')
root = ET.fromstring(fundamental_data)

In [ ]:
def print_xml_tree(element, indent=0):
    """Print the XML tree structure with indentation"""
    # Print current element
    print('  ' * indent + f'<{element.tag}>', end='')
    
    # Print attributes if any
    if element.attrib:
        print(f' {element.attrib}', end='')
        
    # Print text content if it exists and isn't whitespace
    if element.text and element.text.strip():
        print(f' = {element.text.strip()}')
    else:
        print()
        
    # Print all children recursively
    for child in element:
        print_xml_tree(child, indent + 1)

print_xml_tree(root)

<ReportSnapshot> {'Major': '1', 'Minor': '0', 'Revision': '1'}
  <CoIDs>
    <CoID> {'Type': 'RepNo'} = C8279
    <CoID> {'Type': 'CompanyName'} = Tesla Inc
    <CoID> {'Type': 'IRSNo'} = 912197729
    <CoID> {'Type': 'CIKNo'} = 0001318605
    <CoID> {'Type': 'OrganizationPermID'} = 5088024644
  <Issues>
    <Issue> {'ID': '1', 'Type': 'C', 'Desc': 'Common Stock', 'Order': '1'}
      <IssueID> {'Type': 'Name'} = Ordinary Shares
      <IssueID> {'Type': 'Ticker'} = TSLA
      <IssueID> {'Type': 'RIC'} = TSLA.O
      <IssueID> {'Type': 'DisplayRIC'} = TSLA.OQ
      <IssueID> {'Type': 'InstrumentPI'} = 67910050
      <IssueID> {'Type': 'QuotePI'} = 72106022
      <Exchange> {'Code': 'NASD', 'Country': 'USA'} = NASDAQ
      <GlobalListingType> = OSR
      <MostRecentSplit> {'Date': '2022-08-25'} = 3.00003
    <Issue> {'ID': '2', 'Type': 'P', 'Desc': 'Preferred Stock', 'Order': '1'}
      <IssueID> {'Type': 'Name'} = Preference Shares Series B
      <IssueID> {'Type': 'InstrumentPI'} = 7447